In [3]:
from os import urandom

In [31]:
def pad(data: bytes, size: int = 16, mode: str = 'pkcs7') -> bytes:
    """
    modes: pkcs7, x923, iso10126, iso7816.
    """
    if size < 1: raise ValueError('size must be greater than 0')
    diff = size - len(data) % size
    if diff == 0: diff = size
    if mode == 'pkcs7':
        return data + bytes([diff] * diff)
    if mode == 'x923':
        return data + (b'\x00' * (diff - 1)) + bytes([diff])
    if mode == 'iso10126':
        return data + urandom(diff - 1) + bytes([diff])
    if mode == 'iso7816':
        return data + b'\x80' + bytes(b'\x00' * (diff-1))
    if mode == 'zero':
        return data + (b'\x00' * diff)
    raise ValueError('Invalid padding mode.')
    
def unpad(data: bytes, size: int = 16, mode: str = 'pkcs7') -> bytes:
    if size < 1: raise ValueError('size must be greater than 0')
    if len(data) % size != 0: raise ValueError('data is not padded')
    if mode == 'iso7816':
        for i in range(1, size+1):
            if data[-1*i] == 128:
                return data[:-1*i]
            if data[-1*i] != 0:
                raise ValueError("Padding is invalid")
    if mode == 'zero':
        for i in range(1, size+2):
            if data[-1*i] != 0:
                return data[:(-1*i)+1]
        raise ValueError("Padding is invalid")
    length = int(data[-1])
    if length > min(size, len(data)): raise ValueError('Padding is invalid.')
    if mode == 'pkcs7':
        for i in range(1, length+1):
            if data[-1*i] != data[-1]:
                raise ValueError("Padding is invalid")
        return data[:-length]
    if mode == 'iso10126':
        return data[:-length]
    if mode == 'x923':
        for i in range(2, length+1):
            if data[-1*i] != b'\x00':
                raise ValueError("Padding is invalid")
        return data[:-length]
    raise ValueError('Invalid padding mode.')
    